# LLM graph iteration demo

<p style="font-size: 20px; font-weight: bold; font-style: italic;">...without LLM</p>

Anton Antonov   
January 2026

---

## Introduction

This notebook shows how to do iterative refinements of results agent results using a iterative evaluation of an (acyclic) LLM-graph.
The setup uses a deterministic, "toy LLM" that allows for quick and easy demonstration of the overall process. (And it is suitable for debugging.)

---

## Setup

In [1]:
use LLM::Graph;

---

## Define state and a tiny deterministic LLM stub

The stub makes the output repeatable so you can focus on graph behavior.

In [2]:
proto sub toy-llm($form) {*}

multi sub toy-llm(Str:D $draft) {
    toy-llm({critique => '', critique-iteration => 0, revision => $draft, revision-iteration => 0})
}

multi sub toy-llm(%form) {
    if %form<critique-iteration> ≤ %form<revision-iteration> {
        %form<critique> = do given %form<critique-iteration> {
            when $_ == 0  { 'Needs a clearer thesis and one concrete example.' }
            when $_ == 1  { 'Much better. Add a concise closing sentence.' }
            default { "Looks good." }
        }
        %form<critique-iteration> += 1;
        return %form
    }
    %form<revision> ~= "\n[Revision applied: tightened and clarified.]";
    %form<revision-iteration> += 1;
    return %form
}

&toy-llm

----

## Graph nodes

The cycle is: generate -> critique -> revise -> critique -> ... and stops when the critique says "Looks good." or max iterations is reached.

In [3]:
sink my %generation-rules =

    generate => { 
        eval-function => sub ($topic) {
            my $draft = "Draft (iteration 1) on $topic:\n" ~
            "- Thesis: $topic matters.\n" ~ 
            "- Point: Provide one benefit.\n" ~
            "- Example: TBD.\n";
            return $draft;
        }
    }
;

In [4]:
sink my %revision-rules =

    decide => { 
        eval-function => sub ($text) { 
            !(
                $text ~~ Str:D && $text.contains("Looks good") ||
                $text ~~ Map:D && $text<critique>.contains("Looks good")
            )
        } 
    },

    critique => {
        eval-function => sub ($text) {
            return toy-llm($text) if $text ~~ Str:D;
            
            my $form = $text.clone; 
            $form<critique-iteration> += $form<critique-iteration>;

            return toy-llm($form);
        },
        
        test-function => sub ($decide) { $decide.raku.lc.contains('true') }
    },

    revise => {
        eval-function => sub ($text, $critique) {
            my $form = $critique.clone;
            $form = toy-llm($form);
            return $form;
        },
        
        test-function => sub ($critique) { $critique.defined }
    },

    finalize => {
        eval-function => sub ($text, $revise) { $revise.defined ?? $revise !! $text}
    }  
;

---


## Build the graphs

In [5]:
my $g1 = LLM::Graph.new(%generation-rules):!async;
my $g2 = LLM::Graph.new(%revision-rules):!async;

(:$g1, :$g2)

(g1 => LLM::Graph(size => 1, nodes => generate) g2 => LLM::Graph(size => 4, nodes => critique, decide, finalize, revise))

---

## Run and inspect state transitions

In [6]:
$g1.eval({ topic => "why cyclic graphs help with iterative writing" })

LLM::Graph(size => 1, nodes => generate)

In [7]:
my $text = $g1.nodes<generate><result>

Draft (iteration 1) on why cyclic graphs help with iterative writing:
- Thesis: why cyclic graphs help with iterative writing matters.
- Point: Provide one benefit.
- Example: TBD.


In [8]:
$g2.eval({ :$text })

LLM::Graph(size => 4, nodes => critique, decide, finalize, revise)

In [9]:
$g2.nodes<revise>

{eval-function => sub { }, input => [critique text], result => {critique => Needs a clearer thesis and one concrete example., critique-iteration => 1, revision => Draft (iteration 1) on why cyclic graphs help with iterative writing:
- Thesis: why cyclic graphs help with iterative writing matters.
- Point: Provide one benefit.
- Example: TBD.

[Revision applied: tightened and clarified.], revision-iteration => 1}, spec-type => (Callable), test-function => sub { }, test-function-input => [critique], test-function-result => True}

In [10]:
$g1.clear;
$g2.clear;

$g1.eval({ topic => "why cyclic graphs helpe with iterative writin??g" });
my $text = $g1.nodes<generate><result>;

for (^4) -> $iter {
  say '-' x 10, $iter, '-' x 10;
  $g2.eval({:$text});  
  my $revision = $g2.nodes<finalize><result>;
  last if $revision eq $text;
  $text = $revision
}

$text

----------0----------
----------1----------
----------2----------


{critique => Looks good., critique-iteration => 3, revision => Draft (iteration 1) on why cyclic graphs helpe with iterative writin??g:
- Thesis: why cyclic graphs helpe with iterative writin??g matters.
- Point: Provide one benefit.
- Example: TBD.

[Revision applied: tightened and clarified.]
[Revision applied: tightened and clarified.], revision-iteration => 2}

----

## Final output

In [11]:
#% html
my %opts = engine => 'dot', :9graph-size, node-width => 1.2, theme => 'ortho';
[
    generation-graph => $g1.dot(|%opts, :1graph-size, :svg),
    revision-graph => $g2.dot(|%opts, :3graph-size, :svg)
]
==> to-html-table()

generation-graph revision-graph <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- generate -->
 
 generate 
 
 generate 
 
<!-- topic -->
 
 topic 
 
 topic 
 
<!-- topic->generate -->
 
 topic->generate 
 
 
 
 
 
 <?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 14.1.1 (20251213.1925)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- critique -->
 
 critique 
 
 critique 
 
<!-- revise -->
 
 revise 
 
 revise 
 
<!-- critique->revise -->
 
 critique->revise 
 
 
 
<!-- decide -->
 
 decide 
 
 decide 
 
<!-- decide->critique -->
 
 decide->critique 
 
 
 
<!-- finalize -->
 
 finalize 
 
 finalize 
 
<!-- revise->finalize -->
 
 revise->finalize 
 
 
 
<!-- text -->
 
 text 
 
 text 
 
<!-- text->critique -->
 
 text->critique 
 
 
 
<!-- text->decide -->
 
 text->decide 
 
 
 
<!-- text->finalize -->
 
 text->finalize 
 
 
 
<!-- text->revise -->
 
 text->revise